## Load dependencies

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
os.environ["CUDA_VISIBLE_DEVICES"] = '1,2'


## load the position dictionary to locate where to put the code embedding

In [ ]:
dxdict=pickle.load(open('dxpos.pkl','rb'))
meddict=pickle.load(open('medpos.pkl','rb'))

## Load the data, including the doc2vec embedding, bert embedding and raw feature input

In [ ]:
from sklearn.utils import shuffle

## Raw data feature input
optrainx=np.load('optrainx.npy')
optrainy=np.load('optrainy.npy')

## BERT embedding for clinical code
token2emb=np.load('token2emb_bert.npy')
## the edge data, [0] [1] are the connected nodes
opedges=np.load('opedges.npy')

## Doc2vec embedding for clinical code
diagnosis_desc_dict=pickle.load(open('diagnosis_desc_dict.pkl','rb'))
med_desc_dict=pickle.load(open('med_desc_dict.pkl','rb'))

In [ ]:
## number of patients
patient_totest=len(optrainx)

## number of clinical codes that have embeddings
num_codes=len(diagnosis_desc_dict)+len(med_desc_dict)


## use embedding to initialize the node features
embedding_layer = tf.keras.layers.Embedding(num_codes, 64)

## node feature vector for clinical code
opnode_features_part_c=embedding_layer(np.asarray([i for i in range(num_codes)])).numpy()
## node feature vector for encounter and patient
opnode_features_part_ep = np.random.rand(patient_totest,32)

## replace the clinical code embeddings by bert and doc2vec embeddings
for dx in diagnosis_desc_dict:
    opnode_features_part_c[dxdict[dx],:32]=diagnosis_desc_dict[dx]

for med in med_desc_dict:
    opnode_features_part_c[meddict[med]+414,:32]=med_desc_dict[med]

for i in range(562):
    opnode_features_part_c[i,32:]=token2emb[i]

print(opnode_features_part_c.shape,opnode_features_part_ep.shape)


opedges=tf.convert_to_tensor(opedges,dtype=tf.int64)

graph_info = (opnode_features_part_c, opnode_features_part_ep, opedges)

other_features=optrainx[:patient_totest,:5,:]

## Build the input x and y, then split the train and validation set

In [ ]:
opx=[]
opy=[]

for i in range(patient_totest):
    opx.append([j for j in range(num_codes+i*5,num_codes+5+i*5)])

opx=np.asarray(opx)

opy=optrainy[:patient_totest]


trainopx1=opx[:int(patient_totest*0.8)]
trainopx2=optrainx[:int(patient_totest*0.8)]
trainopy=opy[:int(patient_totest*0.8)]

valopx1=opx[int(patient_totest*0.8):]
valopx2=optrainx[int(patient_totest*0.8):]
valopy=opy[int(patient_totest*0.8):]

## Implement Feedforward Network (FFN) Module

In [ ]:
def create_ffn(hidden_units, dropout_rate, name=None):
    fnn_layers = []

    for units in hidden_units:
        fnn_layers.append(layers.BatchNormalization())
        fnn_layers.append(layers.Dropout(dropout_rate))
        fnn_layers.append(layers.Dense(units, activation=tf.nn.relu))

    return keras.Sequential(fnn_layers, name=name)

## Build a Graph Neural Network Model

In [ ]:
class HeteroGraphConvLayer(layers.Layer):
    def __init__(
        self,
        hidden_units,
        dropout_rate=0.2,
        aggregation_type="mean",
        combination_type="add",
        normalize=False,
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)

        self.aggregation_type = aggregation_type
        self.combination_type = combination_type
        self.normalize = normalize

        self.ffn_prepare = create_ffn(hidden_units, dropout_rate)
        self.update_fn = create_ffn(hidden_units, dropout_rate)

    def prepare(self, node_repesentations, weights=None):
        # node_repesentations shape is [num_edges, embedding_dim].
        messages = self.ffn_prepare(node_repesentations)
        #if weights is not None:
        #    messages = messages * tf.expand_dims(weights, -1)
        return messages

    def aggregate(self, node_indices, neighbour_messages, node_repesentations):
        # node_indices shape is [num_edges].
        # neighbour_messages shape: [num_edges, representation_dim].
        # node_repesentations shape is [num_nodes, representation_dim]
        num_nodes = node_repesentations.shape[0]

        aggregated_message = tf.math.unsorted_segment_mean(
                neighbour_messages, node_indices, num_segments=num_nodes
        )

        return aggregated_message

    def update(self, node_repesentations, aggregated_messages):
        # node_repesentations shape is [num_nodes, representation_dim].
        # aggregated_messages shape is [num_nodes, representation_dim].
        # Add node_repesentations and aggregated_messages.
        h = node_repesentations + aggregated_messages
        # Apply the processing function.
        node_embeddings = self.update_fn(h)
        if self.combination_type == "gru":
            node_embeddings = tf.unstack(node_embeddings, axis=1)[-1]

        if self.normalize:
            node_embeddings = tf.nn.l2_normalize(node_embeddings, axis=-1)
        return node_embeddings

    def call(self, inputs):

        node_repesentations, edges = inputs
        # Get node_indices (source) and neighbour_indices (target) from edges.
        node_indices, neighbour_indices = edges[0], edges[1]
        # neighbour_repesentations shape is [num_edges, representation_dim].
        neighbour_repesentations = tf.gather(node_repesentations, neighbour_indices)
        
        # Prepare the messages of the neighbours.
        neighbour_messages = self.prepare(neighbour_repesentations)
        # Aggregate the neighbour messages.
        aggregated_messages = self.aggregate(
            node_indices, neighbour_messages, node_repesentations
        )
        # Update the node embedding with the neighbour messages.
        return self.update(node_repesentations, aggregated_messages)

## To implement LSTM + GNN integrated model with nodes encoding

In [ ]:
class LSTMGNNClassifier(tf.keras.Model):
    def __init__(
        self,
        graph_info,
        hidden_units,
        aggregation_type="sum",
        combination_type="concat",
        dropout_rate=0.2,
        normalize=True,
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)

        # Unpack graph_info to three elements: node_features, edges, and edge_weight.
        clinical_features, enc_pat_features, edges = graph_info
        self.clinical_features = clinical_features
        self.enc_pat_features = enc_pat_features
        self.edges = edges

        # Create a process layer.
        self.clinical_encoder = create_ffn(hidden_units, dropout_rate, name="preprocess")
        self.preprocess = create_ffn(hidden_units, dropout_rate, name="preprocess")
        # Create the first GraphConv layer.
        self.conv1 = HeteroGraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv1",
        )
        
        self.conv2 = HeteroGraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv2",
        )
        
        self.conv3 = HeteroGraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv3",
        )
        
        # Create a postprocess layer.
        self.postprocess = create_ffn(hidden_units, dropout_rate, name="postprocess")
        
        self.lstm_1= tf.keras.layers.LSTM(16,return_sequences=True)
        self.lstm_2= tf.keras.layers.LSTM(16)

        self.lastlayer = layers.Dense(units=8, activation='relu')
        # Create a compute logits layer.
        self.finaloutput = layers.Dense(units=1, activation='sigmoid')

    def call(self, inputs):
        # Preprocess the node_features to produce node representations.
        
        input_node_indices,otherfeatures=inputs[0],inputs[1]
        
        xc = self.clinical_encoder(self.clinical_features)
        xep = self.preprocess(self.enc_pat_features)
        
        x = tf.concat((xc,xep),axis=0)
        
        # Apply the first graph conv layer.
        x1 = self.conv1((x, self.edges))

        x = x1 + x
        
        opedges2=tf.gather(self.edges,[1,0],axis=0)
        
        # Apply the second graph conv layer.
        x2 = self.conv2((x, opedges2))
        # Skip connection.
        x = x2 + x
        
        # Postprocess node embedding.
        # Apply the second graph conv layer.
        x3 = self.conv3((x, self.edges))
        # Skip connection.
        x = x3 + x
        # Postprocess node embedding.
        x = self.postprocess(x)

        # Fetch node embeddings for the input node_indices.
        node_embeddings = tf.gather(x, input_node_indices)
        #step_embeddings=node_embeddings.reshape((input_node_indices.shape[0],5,node_embeddings.shape[-1]))
        concat_embeddings=tf.concat([node_embeddings, otherfeatures], 2)
        lstm_embeddings_1=self.lstm_1(concat_embeddings)
        lstm_embeddings_2=self.lstm_2(lstm_embeddings_1)
        output_embeddings=self.lastlayer(lstm_embeddings_2)
        # Compute logits
        
        return self.finaloutput(output_embeddings)


## To initialize the model


In [ ]:
hidden_units = [32, 32]
learning_rate = 0.01
dropout_rate = 0.5


gnn_model = LSTMGNNEncounterClassifier(
    graph_info=graph_info,
    hidden_units=hidden_units,
    dropout_rate=dropout_rate,
    name="gnnlstm_model",
)
gnn_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[keras.metrics.BinaryAccuracy(name="binary_accuracy")],
)

gnn_model.fit([trainopx1,trainopx2],trainopy,epochs=50,batch_size=128)

## Calculate the best f1 score

In [ ]:
def rod(a,b):
    if a>b:
        return 1
    else:
        return 0
def bestf(predy,testy):
    from sklearn.metrics import recall_score,precision_score,f1_score,roc_auc_score,average_precision_score
    f1s=[]
    for i in range(1,100):
        j=i/100.0
        pren=list(map(lambda x:rod(x,j),predy))
        f1s.append([f1_score(testy,pren),precision_score(testy,pren),recall_score(testy,pren),i])
    
    maxf1=max(f1s)
    auroc=roc_auc_score(testy,predy)
    auprc=average_precision_score(testy,predy)
    
    print('F1 Score:',maxf1[0],' Precision Score:',maxf1[1],' Recall Score:',maxf1[2],'AUROC:',auroc,'Average Precision',auprc)

In [ ]:
bestf(allprevaly,valopy)

## To test the validation performance

In [ ]:
allprevaly=[]
for i in range(10):
    prevaly=gnn_model.predict([valopx1[i*1000:(i+1)*1000],valopx2[i*1000:(i+1)*1000]])
    allprevaly.append(prevaly)

allprevaly=np.concatenate(allprevaly)
bestf(allprevaly,valopy)